In [6]:
import pandas as pd
import numpy as np

import os, glob

In [1]:
TICKETS_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/KPI Reports - 2/POMS in KPI Reports/"
OTHER_TICKETS = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/KPI Reports - 2/linehaul_poms/"

In [32]:
KPI_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/KPI Reports - 2/POMS in KPI Reports/"
OTHER_TICKETS = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/KPI Reports - 2/linehaul_poms/"

kpi_files = glob.glob(os.path.join(OTHER_TICKETS, "*.csv"))# glob.glob(os.path.join(KPI_PATH, "*.csv")) #
df_kpi = pd.concat((pd.read_csv(f, encoding='ISO-8859-1', engine='c', dtype={'Order #': str}, usecols=[
    'Order #',
    'Order Status',
    'Actual Delivery Date',
    'First Offered Date',
    'Client Name',
    'Amount',
    'Approval Value']
    ) for f in kpi_files), ignore_index=True).round(2)
df_kpi_g = df_kpi.groupby('Order #').agg({
    'Order Status': 'first',
    'Actual Delivery Date': 'first',
    'First Offered Date': 'first',
    'Client Name': 'first',
    'Amount': 'sum',
    'Approval Value': 'last'}
    )

In [33]:
orders = df_kpi.loc[(df_kpi['Amount'] == 0) & (df_kpi['Approval Value'] != "Invoice Finalized"), ["Order #", "Approval Value"]]

In [34]:
orders.count()

Order #           0
Approval Value    0
dtype: int64

In [31]:
orders.to_csv("../Product/MissingOrdersAmount.csv")

In [5]:
import pandas as pd

df_kpi = pd.concat((pd.read_csv(f, encoding='ISO-8859-1', engine='c', dtype={'Order #': object},
                                usecols=['Order #', 'Actual Delivery Date', 'Client Name', 'Amount']
                                ) for f in kpi_files), ignore_index=True).drop_duplicates().round(2)

In [7]:
df_kpi[df_kpi['Order #'] == "3216162"]

,Order #,Client Name,Actual Delivery Date,Amount
2024660,3216162,Spot International Inc,44978.33,0.0


In [23]:
REPORT_PATH = "//filesrv/MercuryGate/"
PARQUET_PATH = "//filesrv/MercuryGate/Separated/Backup.parquet"
ORDERS_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/Pull Sheet Data LH Team/"
TICKETS_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/Closing Tickets/"
OUTPUT_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/TMSProduct/"
AUTO_PULL_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/pompy/get_missing_pull_sheets.py"
AUTO_TICKETS_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/pompy/get_missing_closing_tickets.py"
AUTO_DIST_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/pompy/get_missing_dist.py"

In [6]:
df = pd.DataFrame({'A': [np.nan, 'X-Y']})
def convert(lane):
    stops = str(lane).split("-")
    print(stops[0] == 'nan')

df['A'].apply(convert)

True
False


0    None
1    None
Name: A, dtype: object

In [5]:
df = pd.Series([np.nan, 0, 10])
dx = np.where(df <= 0,
              3 / 1,
              3 / df)

In [52]:
data = {
    'I': ['X', 'X', 'X', 'Y', np.nan],
    'P': ['LD1', 'LD1', 'LD2', 'LD1', 'LD3'],
    'C': [100, 100, 100, 70, 0],
    'TLH': [80, 80, 80, 55, 0],
    'TFS': [20, 20, 20, 15, 0],
    'CT': [125, 125, 125, 65, 30]
}
df = pd.DataFrame(data)

In [54]:
df['I'] = df['I'].fillna('')

In [56]:
nu_lo_ag_inv = df.groupby('I')['P'].transform('nunique')
df['C'] = df['C'] / nu_lo_ag_inv
df['TLH'] = df['TLH'] / nu_lo_ag_inv
df['TFS'] = df['TFS'] / nu_lo_ag_inv

In [64]:
df

,I,P,CT,C,TLH,TFS
0,Y;X,LD1,120.0,120.0,95.0,25.0
3,X,LD2,50.0,50.0,40.0,10.0
4,,LD3,30.0,NaN,0.0,0.0


In [57]:
df_gb = df.groupby([
    'P',
    'I'])[[
        'C',
        'TLH',
        'TFS']].max().groupby(level='P').sum()

In [58]:
df = df.drop(columns=['C',
                      'TLH',
                      'TFS'])
df = df.merge(df_gb, on='P')

In [60]:
df['I'] = df.groupby('P')['I'].transform(lambda x: ";".join(set(x)))

In [61]:
df.drop_duplicates(subset=['P'], inplace=True)

In [62]:
df['C'] = df['C'].replace(to_replace=0, value=np.nan)

In [63]:
df['CT'] = df['C'].fillna(df['CT'])

In [2]:
df = pd.read_parquet(r"\\filesrv\MercuryGate\Separated\Backup.parquet")

In [3]:
df.columns

Index(['Primary Reference', 'Pinnacle Manifest No', 'Target Ship (Late)',
       'Target Delivery (Late)', 'Actual Delivery', 'Origin Code',
       'Origin Zip', 'Dest Code', 'Dest Zip', 'Carrier', 'Carrier Total',
       'Invoice Charge', 'Invoice Date', 'Invoice Number',
       'Invoice Total Fuel', 'Invoice Total Line Haul'],
      dtype='object')

In [6]:
df["Target Ship (Late)"] = pd.to_datetime(df["Target Ship (Late)"])
df["Target Delivery (Late)"] = pd.to_datetime(df["Target Delivery (Late)"])
df["Invoice Date"] = pd.to_datetime(df["Invoice Date"])

In [7]:
cutoff_date = '2023-02-01'
selected_refs = df.groupby('Primary Reference').filter(lambda x: (x['Target Ship (Late)'] >= cutoff_date).all())['Primary Reference'].unique()
df = df[df['Primary Reference'].isin(selected_refs)]
df['Invoice Number'] = df['Invoice Number'].fillna('')

In [7]:
df.head()

,Create Date,Primary Reference,Pinnacle Manifest No,Status,Carrier,Carrier Total,Invoice Charge,Invoice Date,Invoice Number,Invoice Total Line Haul,...,Dest Zip,Target Delivery (Early),Target Delivery (Late),Actual Delivery,Departure Date,Cube,Weight,Quantity,Shipment Count,OID
15296,2023-02-08 14:52,LD24043 (Load ID),nan,Delivered,X-FACTOR TRANSPORT INC,2750.0,0.0,2023-02-08,10664,0.0,...,2780,2023-02-02 05:00,2023-02-02 10:00:00,2023-02-02 09:30,2023-02-01 15:00,2639.2,13216.27,137.0,1.0,17603561821.0
15297,2023-02-08 14:52,LD24043 (Load ID),nan,Delivered,X-FACTOR TRANSPORT INC,2750.0,0.0,2023-02-08,10664,0.0,...,2780,2023-02-02 05:00,2023-02-02 10:00:00,2023-02-02 09:30,2023-02-02 09:30,2639.2,13216.27,137.0,1.0,17603561821.0
16089,2023-02-08 15:19,LD24016 (Load ID),nan,Delivered,The World Delivered Inc.,1450.0,1450.0,2023-02-03,14353-50295,1450.0,...,27263,2023-02-02 05:00,2023-02-02 07:00:00,2023-02-02 08:00,2023-02-01 09:00,1.0,1.0,1.0,1.0,17603600688.0
16090,2023-02-08 15:19,LD24016 (Load ID),nan,Delivered,The World Delivered Inc.,1450.0,1450.0,2023-02-03,14353-50295,1450.0,...,27263,2023-02-02 05:00,2023-02-02 07:00:00,2023-02-02 08:00,2023-02-02 08:00,1.0,1.0,1.0,1.0,17603600688.0
16091,2023-02-08 15:19,LD24017 (Load ID),99499,Delivered,The World Delivered Inc.,2125.0,2125.0,2023-02-06,14353-50298,2125.0,...,77385,2023-02-03 09:00,2023-02-03 11:00:00,2023-02-03 09:30,2023-02-01 12:50,800.0,1570.0,1.0,1.0,17603600780.0


In [5]:
df[df['Primary Reference']=="LD27922 (Load ID)"]['Target Ship (Late)'].min()

'2023-04-14 08:00'

In [2]:
df2 = pd.read_csv("../TMSProduct/Product.csv", low_memory=False)

In [3]:
df2[df2['Primary Reference']=="LD27922 (Load ID)"]

,manifest_num,Carrier Total,Carrier,lane,Primary Reference,Leg,Leg Zips,Actual Distance,Leg Instance per Load,Adjusted Distance,...,Actual Delivery,Invoice Date,Invoice Number,Invoice Charge,Invoice Total Line Haul,Invoice Total Fuel,from_to_zip,zips,size,Vendor Pickup?


In [8]:
cols = ['Primary Reference', 'Pinnacle Manifest No', 'Target Ship (Late)',
        'Arrival Date', 'Target Delivery (Late)', 'Actual Delivery', 'Origin Code',
        'Origin Zip', 'Dest Code', 'Dest Zip', 'Carrier', 'Carrier Total',
        'Invoice Charge', 'Invoice Date', 'Invoice Number', 'Invoice Total Line Haul',
        'Invoice Total Fuel']
df = df[cols]

In [9]:
#divide the invoices equally over their loads, no effect if only 1 load
number_of_loads_against_invoice = df.groupby('Invoice Number')['Primary Reference'].transform('nunique')
df['Invoice Charge'] = df['Invoice Charge'].astype(np.float64) / number_of_loads_against_invoice
df['Invoice Total Line Haul'] = df['Invoice Total Line Haul'].astype(np.float64) / number_of_loads_against_invoice
df['Invoice Total Fuel'] = df['Invoice Total Fuel'].astype(np.float64) / number_of_loads_against_invoice

In [10]:
#aggregate costs against reference, summing against unique invoices
df_gb = df.groupby([
    'Primary Reference',
    'Invoice Number'])[[
        'Invoice Charge',
        'Invoice Total Line Haul',
        'Invoice Total Fuel']].max().groupby(level='Primary Reference').sum()
df = df.drop(columns=['Invoice Charge',
                      'Invoice Total Line Haul',
                      'Invoice Total Fuel'])
df = df.merge(df_gb, on='Primary Reference')

In [11]:
#aggregate invoice number to create a list, and aggregate invoice date to get max
df['Invoice Number'] = df.groupby('Primary Reference')['Invoice Number'].transform(lambda x: ";".join(set(x)))
df['Invoice Date'] = df.groupby('Primary Reference')['Invoice Date'].transform('max')

In [12]:
#replace 0s with nans to fillna later
df['Invoice Charge'] = df['Invoice Charge'].replace(to_replace=0, value=np.nan)

In [13]:
#replace carrier total with invoice charge if invoice charge is not na
df['Carrier Total'] = df['Invoice Charge'].fillna(df['Carrier Total'])

In [14]:
df.drop_duplicates(subset=['Primary Reference', 'Pinnacle Manifest No', 'Origin Code',
                           'Dest Code','Target Ship (Late)'], inplace=True, keep='last')

In [15]:
df['Origin Code'].fillna(df['Origin Zip'], inplace=True)
df['Dest Code'].fillna(df['Dest Zip'], inplace=True)

In [16]:
df["from_to"] = df["Origin Code"].astype('str') + " | " + df["Dest Code"].astype('str')
df["from_to_zip"] = df["Origin Zip"].astype('str') + ";" + df["Dest Zip"].astype('str')
df["Code;Zip"] = df["Dest Code"].astype('str') + ";" + df["Dest Zip"].astype('str')
df["OriginCode;Zip"] = df["Origin Code"].astype('str') + ";" + df["Origin Zip"].astype('str')

In [17]:
def create_lane(load):
    rows = df.loc[df['Primary Reference']==load, ["Target Ship (Late)", "Target Delivery (Late)",
                                                    "OriginCode;Zip", "Code;Zip"]]
    targets = pd.concat([rows["Target Ship (Late)"], rows["Target Delivery (Late)"]])
    code_zips = pd.concat([rows["OriginCode;Zip"], rows["Code;Zip"]])
    rows = pd.DataFrame({"Target": targets, "Code": code_zips}).sort_values(by="Target")
    l = ['']; li = ['']
    for x in rows["Code"]:
        if l[-1] != x.split(";")[0]:
            l.append(x.split(";")[0])
            li.append(x.split(";")[1])
    l = l[1:]
    li = li[1:]
    lane = " | ".join(l)
    zips = ";".join(li)
    return lane, zips

df['lane'], df['zips'] = zip(*df['Primary Reference'].apply(create_lane))

In [18]:
def process_manifest(manifest):
    manifest = str(manifest).strip().split(", ")
    size = []
    is_vendor_pickup = []
    for i, j in enumerate(manifest):
        if "VEN" in j.upper() or "PU" in j.upper():
            manifest[i] = 0
            is_vendor_pickup.append("Yes")
            size.append('- 53FT')
        else:
            is_vendor_pickup.append("No")
            if 'ft' in j.lower():
                manifest[i] = ''.join(filter(str.isdigit, j))[:7]
                size.append(j)
            else:
                manifest[i] = ''.join(filter(str.isdigit, j))[:6]
                size.append('- 53FT')
    return manifest, size, is_vendor_pickup

df['Pinnacle Manifest No'], df['size'], df['Vendor Pickup?'] = zip(*df['Pinnacle Manifest No'].apply(process_manifest))

In [19]:
df = df.explode(['Pinnacle Manifest No', 'size', 'Vendor Pickup?'])

In [20]:
df['Pinnacle Manifest No'] = pd.to_numeric(df['Pinnacle Manifest No'], errors='coerce'
                                           ).fillna(0).astype(int)
df = df.rename(columns={'Pinnacle Manifest No': 'manifest_num'})

In [21]:
print(df.shape)

(4617, 25)


In [24]:
# %%
import os
import subprocess


tempdf = pd.DataFrame()
tempdf["manifest_num"] = df["manifest_num"].drop_duplicates().dropna()
tempdf.to_csv(OUTPUT_PATH + "temp.csv", index=False)

# %% [markdown]
# Run the auto pulling scripts to get the Pull Sheet Data and the Closing Tickets.

# %%
subprocess.run(["python", AUTO_PULL_PATH, OUTPUT_PATH], stdout=subprocess.PIPE)
subprocess.run(["python", AUTO_TICKETS_PATH], stdout=subprocess.PIPE)

# %%
os.remove(OUTPUT_PATH + "temp.csv")

KeyboardInterrupt: 

In [25]:
def get_leg(list_stops, delim):
    list_legs = []
    for i in range(len(list_stops)-1):
        list_legs.append(list_stops[i] + delim + list_stops[i+1])
    return list_legs

In [26]:
def revise_from_to(lane, zip_lane, from_to, from_to_zip, load):
    err = []
    revised = [from_to]
    revised_zips = [from_to_zip]
    if from_to not in lane:
        origin, destination = from_to.split(" | ")
        stops = lane.split(" | ")
        zips = zip_lane.split(";")
        try:
            origin_index = stops.index(origin)
            destination_index = origin_index + stops[origin_index:].index(destination) + 1
            revised_stops = stops[origin_index:destination_index]
            revised_zips_list = zips[origin_index:destination_index]
            revised = get_leg(revised_stops, " | ")
            revised_zips = get_leg(revised_zips_list, ";")
        except ValueError:
            err.append(load)
    return revised, revised_zips, err

err_df = pd.DataFrame()
df['revised_from_to'], df['revised_zips'], err_df['Load'] = zip(*df.apply(lambda x: revise_from_to(
    x['lane'], x['zips'], x['from_to'], x['from_to_zip'], x['Primary Reference']), axis=1))

In [27]:
df = df.explode(['revised_from_to', 'revised_zips'])

In [28]:
df['count_leg'] = df.groupby(["Primary Reference", "revised_from_to"]).transform('size')

In [29]:
df.loc[(df['Primary Reference']=="LD27907 (Load ID)"), "count_leg"]

13269    1.0
Name: count_leg, dtype: float64

In [30]:
distance_table = pd.read_csv("C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/POM Level/Helpers/ZipDistances.csv", dtype={'Zip_Code': str})
print(distance_table.head())

           Zip  Distance
0  91752;85043       323
1  85043;85043         0
2  85043;91752       324
3  85043;27263      2102
4  27263;85043      2119


In [31]:
failed_dist = pd.DataFrame()
failed_dist['dist'] = df.loc[~df['revised_zips'].isin(distance_table['Zip']), 'revised_zips']
failed_dist = failed_dist.drop_duplicates().dropna()

In [32]:
# split column into multiple columns by delimiter
import os
import subprocess


if len(failed_dist.index) > 0:
    failed_dist[['zip1', 'zip2']] = failed_dist['dist'].str.split(';', n=1, expand=True).dropna(how='any')
    failed_dist.drop(columns=['dist'], inplace=True)
    failed_dist.to_csv(OUTPUT_PATH+'temp_dist.csv', index=False)
    subprocess.Popen(["python", AUTO_DIST_PATH]).wait()
    os.remove(OUTPUT_PATH + "temp_dist.csv")

In [33]:
def get_distance(zips):
    try:
        # both Zip and leg should be strings
        dist = distance_table.loc[distance_table['Zip'] == zips, 'Distance'].iloc[0]
        failed_dist = None
    except:
        dist = 0
        failed_dist = zips
    return dist, failed_dist

exfailed_dist = pd.DataFrame()
df["zip_distance"], exfailed_dist['exdist'] = zip(*df['revised_zips'].apply(get_distance))

In [34]:
# calculate adjusted distance for each row
df['Adjusted Distance'] = df['zip_distance'] * df['count_leg']

# drop duplicates
df.drop_duplicates(['Primary Reference', 'revised_from_to'], inplace=True)

# group by Primary Reference and transform the sum of adjusted distance
df['Load Total Distance'] = df.groupby(['Primary Reference'])['Adjusted Distance'].transform('sum')

# reset index
df.reset_index(drop=True, inplace=True)

In [35]:
df.columns

Index(['Primary Reference', 'manifest_num', 'Target Ship (Late)',
       'Arrival Date', 'Target Delivery (Late)', 'Actual Delivery',
       'Origin Code', 'Origin Zip', 'Dest Code', 'Dest Zip', 'Carrier',
       'Carrier Total', 'Invoice Date', 'Invoice Number', 'Invoice Charge',
       'Invoice Total Line Haul', 'Invoice Total Fuel', 'from_to',
       'from_to_zip', 'Code;Zip', 'OriginCode;Zip', 'lane', 'zips', 'size',
       'Vendor Pickup?', 'revised_from_to', 'revised_zips', 'count_leg',
       'zip_distance', 'Adjusted Distance', 'Load Total Distance'],
      dtype='object')

In [ ]:
df.drop(columns=['Legs per Load', 'Legs per Load_x', 'Legs per Load_y'], inplace=True)

In [36]:
# calculate number of legs per load
legs_per_load = df.groupby(['Primary Reference']).revised_from_to.nunique().reset_index(name='Legs per Load')

# merge manifests_per_leg with fact_table
df = pd.merge(df, legs_per_load, on=['Primary Reference'])

# allocate leg cost to manifests
df['Leg Cost'] = np.where(df['Load Total Distance'] <= 0,
                          df['Carrier Total'].astype('Float64') / df['Legs per Load'].clip(lower=1),
                          df['Carrier Total'].astype('Float64') * df['Adjusted Distance'] / df['Load Total Distance'])

In [37]:
from glob import glob


print("\nGetting the POMs...")
all_files = glob(os.path.join(ORDERS_PATH, "*.csv"))
df_poms = pd.concat((pd.read_csv(f, encoding='ISO-8859-1', engine='c', dtype={'Manifest No': object,
                                                                              'OrderNo': object},
                                usecols=['Manifest No', 'OrderNo', 'Weight', 'Cu_Ft_']
                                ) for f in all_files), ignore_index=True).drop_duplicates().round(2)
df_pomsg = df_poms.groupby(['Manifest No', 'OrderNo'], as_index=False).agg({'Cu_Ft_': 'sum', 'Weight': 'sum'})
df_pomsg['Count'] = df_poms.groupby(['Manifest No', 'OrderNo'], as_index=False).size().loc[:, 'size']
df_poms = df_pomsg.set_index('OrderNo')


Getting the POMs...


In [38]:
print("\nGetting Closing Tickets...")
kpi_files = glob(os.path.join(TICKETS_PATH, "*.csv"))
df_kpi = pd.concat((pd.read_csv(f, encoding='ISO-8859-1', engine='c', dtype={'Order #': object},
                                usecols=['Order #', 'Actual Delivery Date', 'Client Name', 'Amount']
                                ) for f in kpi_files), ignore_index=True).drop_duplicates().round(2)
df_kpi = df_kpi.groupby('Order #').agg({'Actual Delivery Date': 'first', 'Client Name': 'first', 'Amount': 'sum'})


Getting Closing Tickets...


In [39]:
print("\nMerging df_poms and closing_tickets...")
df_poms = df_poms.join(df_kpi)
df_poms.index.names = ['Order #']
df_poms.reset_index(inplace=True)
print(df_poms.shape)


Merging df_poms and closing_tickets...
(1188649, 8)


In [58]:
df.columns

Index(['manifest_num', 'Primary Reference', 'Target Ship (Late)',
       'Arrival Date', 'Target Delivery (Late)', 'Actual Delivery',
       'Origin Code', 'Origin Zip', 'Dest Code', 'Dest Zip', 'Carrier',
       'Carrier Total', 'Invoice Date', 'Invoice Number', 'Invoice Charge',
       'Invoice Total Line Haul', 'Invoice Total Fuel', 'from_to',
       'from_to_zip', 'Code;Zip', 'OriginCode;Zip', 'lane', 'zips', 'size',
       'Vendor Pickup?', 'revised_from_to', 'revised_zips', 'count_leg',
       'zip_distance', 'Adjusted Distance', 'Load Total Distance',
       'Legs per Load', 'Leg Cost'],
      dtype='object')

In [60]:
df_poms.reset_index(inplace=True, drop=False)
df.reset_index(inplace=True, drop=False)

In [61]:
print("\nMerging fact_table and df_poms...")
df_poms['Manifest No'] = df_poms['Manifest No'].fillna(0).astype(int)
df_poms.set_index('Manifest No', inplace=True)
df['manifest_num'].fillna(0, inplace=True)
df.set_index('manifest_num', inplace=True)
fact_table = df.join(df_poms)
fact_table.index.names = ['manifest_num']
fact_table.reset_index(inplace=True)
print(fact_table.shape)


Merging fact_table and df_poms...
(110788, 41)


In [64]:
fact_table['revised_from_to'].fillna('', inplace=True)

In [65]:
from datetime import datetime


fact_table['Order Delivery Date'] = (pd.TimedeltaIndex(fact_table['Actual Delivery Date'
                                            ], unit='d') + datetime(1899, 12, 30)).strftime('%Y-%m-%d')
fact_table.drop(columns=['Actual Delivery Date'], inplace=True)

In [66]:
fact_table['Manifest Cubes'] = fact_table.groupby(['Primary Reference', 'revised_from_to', 'manifest_num'])['Cu_Ft_'].transform('sum')

In [67]:
fact_table['Leg Cubes'] = fact_table.groupby(['Primary Reference', 'revised_from_to'])['Cu_Ft_'].transform('sum')

In [ ]:
fact_table[fact_table['Primary Reference']=="LD27922 (Load ID)"]

In [44]:
fact_table.columns

Index(['manifest_num', 'Primary Reference', 'Target Ship (Late)',
       'Arrival Date', 'Target Delivery (Late)', 'Actual Delivery',
       'Origin Code', 'Origin Zip', 'Dest Code', 'Dest Zip', 'Carrier',
       'Carrier Total', 'Invoice Date', 'Invoice Number', 'Invoice Charge',
       'Invoice Total Line Haul', 'Invoice Total Fuel', 'from_to',
       'from_to_zip', 'Code;Zip', 'OriginCode;Zip', 'lane', 'zips', 'size',
       'Vendor Pickup?', 'revised_from_to', 'revised_zips', 'count_leg',
       'zip_distance', 'Adjusted Distance', 'Load Total Distance',
       'Legs per Load', 'Leg Cost', 'Order #', 'Cu_Ft_', 'Weight', 'Count',
       'Client Name', 'Amount', 'Order Delivery Date', 'Manifest Cubes',
       'Leg Cubes'],
      dtype='object')

In [64]:
fact_table.drop(columns=['Orders Per Manifest'], inplace=True)

In [69]:
# calculate number of manifests per leg
manifests_per_leg = fact_table.groupby(['Primary Reference', 'revised_from_to']).manifest_num.nunique().reset_index(name='Manifests Per Leg')

# merge manifests_per_leg with fact_table
fact_table = pd.merge(fact_table, manifests_per_leg, on=['Primary Reference', 'revised_from_to'])

# allocate leg cost to manifests
fact_table['Manifest Cost'] = np.where(fact_table['Leg Cubes'] <= 0,
                                       fact_table['Leg Cost'].astype('Float64') / fact_table['Manifests Per Leg'].clip(lower=1),
                                       fact_table['Leg Cost'].astype('Float64') * fact_table['Manifest Cubes'] / fact_table['Leg Cubes'])

In [71]:
# calculate number of manifests per leg
orders_per_manifest = fact_table.groupby(['Primary Reference', 'revised_from_to', 'manifest_num'])['Order #'].nunique().reset_index(name='Orders Per Manifest')

# merge orders_per_leg with fact_table
fact_table = pd.merge(fact_table, orders_per_manifest, on=['Primary Reference', 'revised_from_to', 'manifest_num'])

# allocate leg cost to manifests
fact_table['POM Cost'] = np.where(fact_table['Manifest Cubes'] <= 0,
                                  fact_table['Manifest Cost'].astype('Float64') / fact_table['Orders Per Manifest'].clip(lower=1),
                                  fact_table['Manifest Cost'].astype('Float64') * fact_table['Cu_Ft_'] / fact_table['Manifest Cubes'])

In [73]:
fact_table = fact_table.rename(columns={'revised_from_to': 'Leg', 'revised_zips': 'Leg Zips',
                              'count_leg': 'Leg Instance per Load', 'Cu_Ft_': 'Order Cubes',
                              'zip_distance': 'Actual Distance'})

In [74]:
fact_table = fact_table.drop(columns=["Code;Zip", "OriginCode;Zip"])

In [76]:
fact_table['Carrier Total'] = fact_table['Carrier Total'].astype('Float64')

In [80]:
fact_table['POM Cost'].sum()

11626473.864999859

In [81]:
fact_table.dtypes

manifest_num                        int32
index                               int64
Primary Reference                  object
Target Ship (Late)         datetime64[ns]
Arrival Date                       object
Target Delivery (Late)     datetime64[ns]
Actual Delivery                    object
Origin Code                        object
Origin Zip                         object
Dest Code                          object
Dest Zip                           object
Carrier                            object
Carrier Total                     Float64
Invoice Date               datetime64[ns]
Invoice Number                     object
Invoice Charge                    float64
Invoice Total Line Haul           float64
Invoice Total Fuel                float64
from_to                            object
from_to_zip                        object
lane                               object
zips                               object
size                               object
Vendor Pickup?                    

In [82]:
# %%
if os.path.exists(OUTPUT_PATH + "Product.csv"):
    old_data = pd.read_csv(OUTPUT_PATH + "Product.csv")
    references = set(fact_table["Primary Reference"])
    old_data = old_data[~old_data["Primary Reference"].isin(references)]
    final = pd.concat([old_data, fact_table])
else:
    final = fact_table

# %%
final.to_csv(OUTPUT_PATH + "Product.csv", index=False)

C:\Users\shahmir.tariq\AppData\Local\Temp\ipykernel_1944\1516417079.py:3: DtypeWarning: Columns (32,33,34,35,36,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  old_data = pd.read_csv(OUTPUT_PATH + "Product.csv")


In [2]:
import pandas as pd

df = pd.read_csv('../TMSProduct/Product.csv')

C:\Users\shahmir.tariq\AppData\Local\Temp\ipykernel_7748\2746880416.py:3: DtypeWarning: Columns (28,29,30,31,32,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../TMSProduct/Product.csv')


In [3]:
df.columns

Index(['manifest_num', 'Carrier Total', 'Carrier', 'lane', 'Primary Reference',
       'Leg', 'Leg Zips', 'Actual Distance', 'Leg Cost',
       'Target Delivery (Late)', 'from_to', 'Order #', 'Order Cubes', 'Weight',
       'Count', 'Order Delivery Date', 'Client Name', 'Amount', 'Origin Code',
       'Dest Code', 'Origin Zip', 'Dest Zip', 'Manifest Cubes', 'Leg Cubes',
       'Manifests Per Leg', 'Manifest Cost', 'Orders Per Manifest', 'POM Cost',
       'Target Ship (Late)', 'Arrival Date', 'Actual Delivery', 'Invoice Date',
       'Invoice Number', 'Invoice Charge', 'Invoice Total Line Haul',
       'Invoice Total Fuel', 'from_to_zip', 'zips', 'size', 'Vendor Pickup?'],
      dtype='object')

In [4]:
df2 = pd.read_csv('../Others/ProductBeforeAppendingOldData.csv')

In [5]:
df2.columns

Index(['manifest_num', 'Primary Reference', 'Target Ship (Late)',
       'Arrival Date', 'Target Delivery (Late)', 'Actual Delivery',
       'Origin Code', 'Origin Zip', 'Dest Code', 'Dest Zip', 'Carrier',
       'Carrier Total', 'Invoice Date', 'Invoice Number', 'Invoice Charge',
       'Invoice Total Line Haul', 'Invoice Total Fuel', 'from_to',
       'from_to_zip', 'Code;Zip', 'OriginCode;Zip', 'lane', 'zips', 'size',
       'Vendor Pickup?', 'Leg', 'Leg Zips', 'Leg Instance per Load',
       'Actual Distance', 'Adjusted Distance', 'Load Total Distance',
       'Legs per Load', 'Leg Cost', 'Order #', 'Order Cubes', 'Weight',
       'Count', 'Client Name', 'Amount', 'Order Delivery Date',
       'Manifest Cubes', 'Leg Cubes', 'Manifests Per Leg', 'Manifest Cost',
       'Orders Per Manifest', 'POM Cost'],
      dtype='object')

In [6]:
set(df2.columns).difference(df.columns)

{'Adjusted Distance',
 'Code;Zip',
 'Leg Instance per Load',
 'Legs per Load',
 'Load Total Distance',
 'OriginCode;Zip'}

In [15]:
df['Primary Reference'].dtype == df2['Primary Reference'].dtype

True

In [14]:
df2['Primary Reference'] = df2['Primary Reference'].astype('str')

In [ ]:
# %%

# calculate adjusted distance for each row
df['Adjusted Distance'] = df['zip_distance'] * df['count_leg']

# group by Primary Reference and transform the sum of adjusted distance
df['Load Total Distance'] = df.groupby(['Primary Reference'])['Adjusted Distance'].transform('sum')

# reset index
df.reset_index(drop=True, inplace=True)

# %%
# calculate number of legs per load
legs_per_load = df.groupby(['Primary Reference']).revised_from_to.nunique().reset_index(name='Legs per Load')

# merge manifests_per_leg with fact_table
df = pd.merge(df, legs_per_load, on=['Primary Reference'])

# allocate leg cost to manifests
df['Leg Cost'] = np.where(df['Load Total Distance'] <= 0,
                          df['Carrier Total'] / df['Legs per Load'].clip(lower=1),
                          df['Carrier Total'] * df['Adjusted Distance'] / df['Load Total Distance'])


In [69]:
import pandas as pd
import re
import numpy as np

df = pd.DataFrame({'A': ['X', 'X', 'X', 'Y'], 'B': [24, 24, 32, 10],
                   'C': ['A | B', 'A | B', 'B | C', 'C'], 'D': [1, 1, 2, 1], 'E': [100, 100, 100, 150],
                   'M': [1, 2, 3, 4]})

In [70]:
df['Adjusted Distance'] = df['B'] * df['D']

In [71]:
df.drop_duplicates(subset=['A', 'M', 'C'], inplace=True)

In [72]:
load_distance = df.drop_duplicates(['A', 'C']).groupby(['A'])['Adjusted Distance'].sum().reset_index(name='Load Total Distance')

In [73]:
df = pd.merge(df, load_distance, on=['A'])

In [74]:
df.reset_index(drop=True, inplace=True)

In [75]:
df

,A,B,C,D,E,M,Adjusted Distance,Load Total Distance
0,X,24,A | B,1,100,1,24,88
1,X,24,A | B,1,100,2,24,88
2,X,32,B | C,2,100,3,64,88
3,Y,10,C,1,150,4,10,10


In [52]:
legs_per_load = df.groupby(['A']).C.nunique().reset_index(name='Legs per Load')

In [53]:
df = pd.merge(df, legs_per_load, on=['A'])

In [54]:
df['Leg Cost'] = np.where(df['Load Total Distance'] <= 0,
                          df['E'] / df['Legs per Load'].clip(lower=1),
                          df['E'] * df['Adjusted Distance'] / df['Load Total Distance'])

In [55]:
df

,A,B,C,D,E,Adjusted Distance,Load Total Distance,Legs per Load,Leg Cost
0,X,24,A | B,1,100,24,88,2,27.272727
1,X,32,B | C,2,100,64,88,2,72.727273
2,Y,10,C,1,150,10,10,1,150.000000


In [5]:
escaped_search_strings = df['C'].apply(re.escape)
search_pattern = '|'.join(escaped_search_strings)

In [6]:
df['Count'] = df['B'].str.count(search_pattern)

print(df)

   A                  B      C  Count
0  X  A | B | C | A | B  A | B      3
1  X  A | B | C | A | B  B | C      3
2  Y              C | B      C      1


In [19]:
import pandas as pd
import re

# Create a sample dataframe
df = pd.DataFrame({'Key': ['A', 'A', 'B', 'B'],
                   'String': ['A | B | C | A | B', 'A | B | C | A | B', 'C | D | E', 'C | D | E'],
                   'SearchString': ['A | B', 'A | B', 'C | F', '']})



In [20]:
# Function to count occurrences of search string
def count_occurrences(group):
    search_string = group['SearchString'].iloc[0]
    if not search_string:
        return 0
    search_string_escaped = re.escape(search_string)
    return group['String'].str.count(search_string_escaped).max()

# Apply function to each group
result = df.groupby(['Key', 'SearchString']).apply(count_occurrences)

print(result)


Key  SearchString
A    A | B           2
B                    0
     C | F           0
dtype: int64


In [22]:
result.name = 'Count'

In [23]:
merged_df = df.merge(result, on=['Key', 'SearchString'], how='left')

In [24]:
merged_df

,Key,String,SearchString,Count
0,A,A | B | C | A | B,A | B,2
1,A,A | B | C | A | B,A | B,2
2,B,C | D | E,C | F,0
3,B,C | D | E,,0


In [37]:
failed_dist = pd.DataFrame({'zip': ['xyz;abc;234', 'xbc']})

In [41]:
failed_dist[['zip1', 'zip2']] = failed_dist['zip'].str.extract(r'([^;]*);(.*)', expand=True)


In [42]:
failed_dist

,zip,zip1,zip2
0,xyz;abc;234,xyz,abc;234
1,xbc,NaN,NaN


In [43]:
failed_dist.dropna(how='any')

,zip,zip1,zip2
0,xyz;abc;234,xyz,abc;234


In [1]:
origin, destination = "X | Y".split(" | ")

In [15]:
from itertools import product


def find_start_stop(start, stop):
        cross_join = product(start, stop) # cross join the two lists
        path = 99 # default value of path from one index to another (not miles)
        result = () # default value

        for join in cross_join:
            diff = join[1] - join[0]
            # if the current different between two indices is less than path, update path
            if 0 < diff < path:
                path = diff
                result = join

        return result[0], result[1]

In [21]:
def get_leg(list_stops, delim):
    list_legs = []
    if len(list_stops) == 1:
        list_stops.append(list_stops[0])
    for i in range(len(list_stops)-1):
        list_legs.append(list_stops[i] + delim + list_stops[i+1])
    return list_legs

In [35]:
stops = ['x', 'y', 'z', 'y', 'x']
from_to = ''
revised = [from_to]
try:
    origin, destination = from_to.split(" | ")
    origin_indices = [i for i, x in enumerate(stops) if x == origin]
    dest_indices = [i for i, x in enumerate(stops) if x == destination]
    start, stop = find_start_stop(origin_indices, dest_indices)
    revised_stops = stops[start:stop+1]
    revised = get_leg(revised_stops, " | ")
except (IndexError, ValueError):
    pass
print(revised)
#destination_index = origin_index + stops[origin_index:].index(destination) + 1
#revised_stops = stops[origin_index:destination_index]

['']


In [5]:
import pandas as pd

df1 = pd.DataFrame(
    {
        "A": [1, 1, 2, 3],
        "B": [0, 1, 3, 5]
    }
)

df2 = pd.DataFrame(
    {
        "A": [1, 0, 3, 5],
        "B": [1, 2, 4, 8]
    }
)
dff = pd.concat(
    [ df1.loc[~df1["A"].isin(df2["A"])], df2 ],
    ignore_index=True
)

In [6]:
dff

,A,B
0,2,3
1,1,1
2,0,2
3,3,4
4,5,8


In [1]:
from glob import glob
import os

import pandas as pd

In [3]:
TICKETS_PATH = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/KPI Reports - 2/POMS in KPI Reports/"
OTHER_TICKETS = "C:/OneDrive - Metropolitan Warehouse/Vendor Control/Data Files/KPI Reports - 2/linehaul_poms/"

In [4]:
print("\nGetting Closing Tickets...")
# get the CSVs in the main tickets folder 
tp_files = glob(os.path.join(TICKETS_PATH, "*.csv"))
# create a dataframe using the CSVs
df_tp = pd.concat(
    (pd.read_csv(f, encoding='ISO-8859-1', engine='c', dtype={'Order #': object},
                               usecols=[
                                   'Order #',
                                   'Order Status',
                                   'Actual Delivery Date',
                                   'First Offered Date',
                                   'Client Name',
                                   'PickupHub',
                                   'DeliveryHub',
                                   'Amount',
                                   'Approval Value'
                                ]
    ) for f in tp_files), ignore_index=True).round(2)
# get the CSVs in the other tickets folder (the linehaul_poms one)
ot_files = glob(os.path.join(OTHER_TICKETS, "*.csv"))
# create a dataframe using the CSVs
df_ot = pd.concat(
    (pd.read_csv(f, encoding='ISO-8859-1', engine='c', dtype={'Order #': object},
                                usecols=[
                                   'Order #',
                                   'Order Status',
                                   'Actual Delivery Date',
                                   'First Offered Date',
                                   'Client Name',
                                   'PickupHub',
                                   'DeliveryHub',
                                   'Amount',
                                   'Approval Value'
                                ]
    ) for f in ot_files), ignore_index=True).round(2)
# append the two dataframes, taking only those orders from main folder which are
#   not appearing in the linehaul_poms one
df_kpi = pd.concat(
    [ df_tp.loc[~df_tp["Order #"].isin(df_ot["Order #"])], df_ot ],
    ignore_index=True
)
# get rid of the temp dfs made
del df_tp
del df_ot
# groupby Order #
df_kpi = df_kpi.groupby('Order #').agg(
    {
        'Order Status': 'last',
        'Actual Delivery Date': 'last',
        'First Offered Date': 'last',
        'Client Name': 'last',
        'PickupHub': 'last',
        'DeliveryHub': 'last',
        'Amount': 'sum',
        'Approval Value': 'last'
    }
)


Getting Closing Tickets...


In [12]:
df_kpi.loc[df_kpi.index == "360119"]

,Order Status,Actual Delivery Date,First Offered Date,Client Name,PickupHub,DeliveryHub,Amount,Approval Value
Order #,,,,,,,,
